#  Pipeline Silver — Pedidos (Delta Lake)

👨‍💻 **Autor:** Lucas Sousa Santos Oliveira**  
🎯 **Objetivo:** Refino, padronização, governança e **carga incremental robusta** de dados de *Pedidos* para consumo analítico e downstream (Camada Gold).

---

## 🌟 O que você vai encontrar neste README

- **Por que Silver?**: o papel da camada no Data Lakehouse  
- **UPSERT com `MERGE INTO`** (destaque): por que é mais robusto que `APPEND + deduplicação`  
- **CTEs + LEFT JOIN**: legibilidade, organização e preservação total de pedidos  
- **Documentação (governança)**: comentários em tabela/colunas para usuários finais  
- **Boas práticas**: particionamento, Z-ORDER, VACUUM, idempotência, schema evolution  
- **Exemplos de código**: SQL e PySpark prontos para uso

---

## 🎯 Missão da Camada Silver

A **Silver** transforma o *raw* da Bronze em dados **limpos, padronizados e governados**, prontos para consumo.  
No caso de *Pedidos*, as entregas-chave são:

- 🧹 **Qualidade**: deduplicação por `PedidoID`, filtragem de chaves nulas, padronização de nomes/tipos.  
- 🛡 **Governança**: documentação (comentários), metadados e *lineage* explícitos.  
- 🔁 **Incrementalidade robusta**: **UPSERT com `MERGE INTO`** para atualizar/insertar sem retrabalho.  
- ⚡ **Performance**: Repartition + `OPTIMIZE ZORDER` + `VACUUM` no Delta Lake.  

---

## 🧭 Visão de Arquitetura

```mermaid
flowchart TD
    A[🥉 Bronze.Pedidos] --> B[🧹 Limpeza & Padronização via CTEs]
    B --> C[🔗 LEFT JOIN com Estabelecimentos]
    C --> D[🧠 Enriquecimento & Metadados]
    D --> E[🔁 MERGE INTO Silver (UPSERT)]
    E --> F[🛡 Comentários e Catálogo / Metastore]
    F --> G[⚙️ OPTIMIZE + Z-ORDER + VACUUM]
    G --> H[🥇 Pronto para Gold & BI]
```

---

## 🔁 Por que **MERGE INTO (UPSERT)** é superior ao `APPEND + deduplicação`?

| Critério                     | `APPEND + deduplicação posterior`                 | **MERGE INTO (UPSERT)** 🚀 |
|-----------------------------|---------------------------------------------------|----------------------------|
| **Consistência**            | Risco de *races* e duplicatas temporárias         | **ACID**: insere/atualiza atômico |
| **Custo de processamento**  | Dedup a cada execução aumenta custo                | Processa somente mudanças   |
| **Simplicidade**            | Fluxo com múltiplas etapas de correção            | Uma única operação declarativa |
| **Idempotência**            | Difícil de garantir em reprocessos                | Nativamente idempotente     |
| **Escalabilidade**          | Degrada com volume                                | Escala melhor               |

> **Conclusão**: `MERGE INTO` é a abordagem **profissional** para cargas incrementais no Delta Lake.

---

## 🧩 CTEs + LEFT JOIN (legibilidade e preservação de pedidos)

**CTEs** (Common Table Expressions) tornam o pipeline **mais legível e modular**.  
O **LEFT JOIN** garante que **todos os pedidos da Bronze** sejam preservados **mesmo sem correspondência** na dimensão/estabelecimentos (campos do estabelecimento ficam nulos).

```sql
-- 👇 Exemplo SQL completo usando CTEs + LEFT JOIN e MERGE INTO (UPSERT)

WITH bronze_pedidos AS (
  SELECT
    PedidoID,
    EstabelecimentoID,
    Produto,
    CAST(quantidade_vendida AS INT) AS QuantidadeVendida,
    CAST(Preco_Unitario AS DECIMAL(18,2)) AS PrecoUnitario,
    CAST(data_venda AS DATE)            AS DataVenda,
    data_ingestao
  FROM delta.`dbfs:/FileStore/Ampev/tables/bronze/pedidos`
  WHERE PedidoID IS NOT NULL AND TRIM(PedidoID) != ''
),

bronze_estabelecimentos AS (
  SELECT
    EstabelecimentoID,
    Nome   AS NomeEstabelecimento,
    Local  AS LocalEstabelecimento
  FROM delta.`dbfs:/FileStore/Ampev/tables/bronze/estabelecimentos`
),

pedidos_enriquecidos AS (
  SELECT
    p.PedidoID,
    p.EstabelecimentoID,
    p.Produto,
    p.QuantidadeVendida,
    p.PrecoUnitario,
    p.DataVenda,
    e.NomeEstabelecimento,
    e.LocalEstabelecimento,
    current_timestamp() AS data_processamento
  FROM bronze_pedidos p
  LEFT JOIN bronze_estabelecimentos e
    ON p.EstabelecimentoID = e.EstabelecimentoID     -- LEFT JOIN preserva todos os pedidos
)

MERGE INTO silver.pedidos AS tgt
USING pedidos_enriquecidos AS src
ON tgt.PedidoID = src.PedidoID
WHEN MATCHED THEN UPDATE SET
  tgt.EstabelecimentoID   = src.EstabelecimentoID,
  tgt.Produto             = src.Produto,
  tgt.QuantidadeVendida   = src.QuantidadeVendida,
  tgt.PrecoUnitario       = src.PrecoUnitario,
  tgt.DataVenda           = src.DataVenda,
  tgt.NomeEstabelecimento = src.NomeEstabelecimento,
  tgt.LocalEstabelecimento= src.LocalEstabelecimento,
  tgt.data_processamento  = src.data_processamento
WHEN NOT MATCHED THEN INSERT *
;
```

> ✅ *Boas práticas aplicadas*: CTEs nomeadas, *typing* explícito, enriquecimento controlado, **UPSERT transacional**.

---

## 🗒️ Governança: **Documentação com Comentários** (Tabela & Colunas)

Documentar a tabela e suas colunas é **crucial** para analistas e cientistas de dados entenderem **origem, propósito e estrutura**.  
Use **comentários** diretamente no catálogo (Metastore).

```sql
-- Criação (ou garantia) da tabela Silver com comentários completos
CREATE TABLE IF NOT EXISTS silver.pedidos (
  PedidoID              STRING  COMMENT 'Identificador único do pedido (chave de negócio)',
  EstabelecimentoID     STRING  COMMENT 'Chave do estabelecimento de origem do pedido',
  Produto               STRING  COMMENT 'Descrição do produto vendido',
  QuantidadeVendida     INT     COMMENT 'Quantidade de itens vendidos no pedido',
  PrecoUnitario         DECIMAL(18,2) COMMENT 'Preço unitário do produto (BRL)',
  DataVenda             DATE    COMMENT 'Data da venda no fuso padrão do Lakehouse',
  NomeEstabelecimento   STRING  COMMENT 'Nome do estabelecimento associado (pode ser nulo)',
  LocalEstabelecimento  STRING  COMMENT 'Localização do estabelecimento (pode ser nulo)',
  data_processamento    TIMESTAMP COMMENT 'Timestamp do processamento na Silver (auditoria)'
)
USING DELTA
LOCATION 'dbfs:/FileStore/Ampev/tables/silver/pedidos'
COMMENT 'Fato de Pedidos (Silver): dados limpos, padronizados, documentados e prontos para consumo analítico.'
TBLPROPERTIES (
  'quality' = 'silver',
  'owner'   = 'dados@empresa.com',
  'pii'     = 'false'
);

-- Exemplo de documentação adicional (quando a tabela já existe)
COMMENT ON TABLE silver.pedidos IS 'Fato de Pedidos (Silver) — refinado a partir da Bronze, com UPSERT via MERGE.';
COMMENT ON COLUMN silver.pedidos.QuantidadeVendida IS 'Quantidade de itens; sempre inteiro >= 0';
```

> 🧭 **Dica**: `DESCRIBE EXTENDED silver.pedidos` exibe os comentários no Metastore.

---

## 🧪 PySpark (equivalente) — leitura, enriquecimento e UPSERT

```python
from pyspark.sql.functions import current_timestamp, col
from delta.tables import DeltaTable

bronze_pedidos = spark.read.format("delta").load("dbfs:/FileStore/Ampev/tables/bronze/pedidos")
bronze_estabs  = spark.read.format("delta").load("dbfs:/FileStore/Ampev/tables/bronze/estabelecimentos")

# Limpeza/typing básico
bronze_pedidos = (
    bronze_pedidos
      .dropDuplicates(["PedidoID"])
      .filter("PedidoID IS NOT NULL AND TRIM(PedidoID) != ''")
      .withColumn("QuantidadeVendida", col("quantidade_vendida").cast("int"))
      .withColumn("PrecoUnitario", col("Preco_Unitario").cast("decimal(18,2)"))
      .withColumnRenamed("data_venda", "DataVenda")
      .select("PedidoID","EstabelecimentoID","Produto","QuantidadeVendida","PrecoUnitario","DataVenda")
)

# LEFT JOIN para preservar todos os pedidos
enriquecido = (
    bronze_pedidos.alias("p")
    .join(bronze_estabs.selectExpr(
        "EstabelecimentoID",
        "Nome as NomeEstabelecimento",
        "Local as LocalEstabelecimento"
    ).alias("e"), on=col("p.EstabelecimentoID")==col("e.EstabelecimentoID"), how="left")
    .withColumn("data_processamento", current_timestamp())
)

silver_path = "dbfs:/FileStore/Ampev/tables/silver/pedidos"
if DeltaTable.isDeltaTable(spark, silver_path):
    tgt = DeltaTable.forPath(spark, silver_path)
    (tgt.alias("tgt")
        .merge(enriquecido.alias("src"), "tgt.PedidoID = src.PedidoID")
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute())
else:
    (enriquecido.write.format("delta")
        .option("mergeSchema", "true")
        .mode("overwrite")
        .save(silver_path))
```

---

## ⚙️ Otimização & Manutenção (Delta Lake)

```sql
OPTIMIZE silver.pedidos ZORDER BY (PedidoID);
VACUUM silver.pedidos RETAIN 168 HOURS;
```

- `OPTIMIZE + ZORDER`: melhora leitura por `PedidoID` (arquivos compactados & localizar rápido).  
- `VACUUM`: remove arquivos órfãos; reduz custo; mantém histórico controlado.

---

## ✅ Checklist de *Production-Readiness*

- [x] **UPSERT transacional** com `MERGE INTO` (idempotência e ACID)  
- [x] **CTEs + LEFT JOIN** para legibilidade e preservação total de pedidos  
- [x] **Documentação**: comentários em tabela/colunas no catálogo  
- [x] **Schema evolution** habilitado (`mergeSchema`)  
- [x] **Particionamento & Repartition** para escrita/consulta eficientes  
- [x] **OPTIMIZE + ZORDER + VACUUM** aplicados  
- [x] **Pronto para Gold/BI** com semântica clara e governança

---

## 🧠 Conclusão

Este pipeline Silver evidencia **senioridade de engenharia de dados**:  
- Utiliza **MERGE INTO (UPSERT)** como estratégia **profissional** de incrementalidade.  
- Aplica **CTEs e LEFT JOIN** para modularidade e **preservação total** da base de pedidos.  
- Implementa **governança de dados** com **comentários** e metadados úteis a usuários finais.  
- Garante **performance** e **custo-eficiência** com *Delta Lake best practices*.  

> **Resultado**: dados **limpos, documentados e prontos** para análises de alto impacto na camada Gold.